In [8]:
import math

def emo_lev(arousal,valence):    #Function that returns emotion and intensity
    arousal -= 5
    valence -= 5
    r = 5
    x, y = valence, arousal
    dist = (x**2+y**2)**0.5

    #Calc theta in radian
    theta = math.atan(y/x)

    #Convert theta from rad to deg
    theta = 180 * theta/math.pi

    def lev(dist):
        if(dist<1.67):
            return "Low"
        if(dist>1.67 and dist<3.34):
            return "Medium"
        else:
            return "High"
        
    #Check which intensity
    levl = lev(dist)


    def conv_emo(theta):
        if(x>0 and y>0):  #1st Quadrant
            if theta <= 90:
                if(theta<45):
                    return "Happy",1.1,theta  #1st Quad, 1st half
                else:
                    return "Excited",1.2,theta #1st Quad, 2nd half
        else:
            if(x>0 and y<0): #4
                theta+=360
                quad = 4
                if theta <= 360:
                    if(theta<315):
                        return "Calm",4.1,theta
                    else:
                        return "Content",4.2,theta
            else:
                if(x<0 and y>0): #2
                    theta+=180
                    quad = 2
                    if theta <= 180:
                        if(theta<135):
                            return "Angry",2.1,theta
                        else:
                            return "Afraid",2.2,theta
                else:
                    if(x<0 and y<0):
                       theta+=180
                       if theta <= 270:
                            if(theta<225):
                                return "Sad",3.1,theta
                            else:
                                return "Depressed",3.2,theta
                       
                   
    #print("Theta before conversion: {}".format(theta))
    emo,quad,theta = conv_emo(theta)
    #print("Distance = {} \nTheta after conversion = {} \nquadrant = {} ".format(dist,theta,quad))
    return emo,levl,theta,quad    #END of emo_lev

#Input the valence, arousal values here.
valence_x, arousal_y = 2,3
emotion, level, theta,quadrant = emo_lev(arousal_y,valence_x)

#Optional printing. Just return the values you need if you want.
print("Emotion    : "+emotion)
print("Intensity  : "+level)
print("Theta      : {} degree".format("%.2f"%theta))
print("Quadrant   : {}".format(quadrant))

Emotion    : Sad
Intensity  : High
Theta      : 213.69 degree
Quadrant   : 3.1


In [9]:
from models.lstm import *
from dataset.DEAP_DATASET import ModularDeapDataset
from torch.utils.data import DataLoader
import torch
from util.train import select_last_elm

DATA_SET_PATH = 'dataset/'
deap_val = ModularDeapDataset(DATA_SET_PATH, train=False)
deap_train = ModularDeapDataset(DATA_SET_PATH, train=True)
train_loader = DataLoader(deap_train, shuffle=True, batch_size=1)
val_loader = DataLoader(deap_val, shuffle=True, batch_size=1)

INPUT_SIZE = 40
HIDDEN_SIZE1 = 64
HIDDEN_SIZE2 = 32
OUTPUT_SIZE = 4

MDL_PATH = 'models/saved_weights/tcn_v0.pth'

model = EEG_TCN(HIDDEN_SIZE1, 1)

model.load_state_dict(torch.load(MDL_PATH, map_location="cpu"))
model.eval()

with torch.no_grad():
    for i, (signal, label) in enumerate(train_loader):
        hidden = model.initHidden()
        output = model(signal, hidden)
        output = select_last_elm(output, "cpu")
        output = output.numpy()
        label = label.numpy()
        print("Predicted:", output, "Actual Label:", label)
        
        emotion, level, theta,quadrant = emo_lev(output[0][1],output[0][0])
        print("PREDICTED",output)
        print("\tEmotion    : "+emotion)
        print("\tIntensity  : "+level)
        print("\tTheta      : {} degree".format("%.2f"%theta))
        print("\tQuadrant   : {}".format(quadrant))
        
        emotion, level, theta,quadrant = emo_lev(label[0][1],label[0][0])
        print("ACTUAL",label)
        print("\tEmotion    : "+emotion)
        print("\tIntensity  : "+level)
        print("\tTheta      : {} degree".format("%.2f"%theta))
        print("\tQuadrant   : {}".format(quadrant))


RuntimeError: Error(s) in loading state_dict for MiniLSTM:
	Missing key(s) in state_dict: "lstm1.weight_ih_l0", "lstm1.weight_hh_l0", "lstm1.bias_ih_l0", "lstm1.bias_hh_l0", "fc.weight", "fc.bias". 
	Unexpected key(s) in state_dict: "tcn.network.0.conv1.bias", "tcn.network.0.conv1.weight_g", "tcn.network.0.conv1.weight_v", "tcn.network.0.conv2.bias", "tcn.network.0.conv2.weight_g", "tcn.network.0.conv2.weight_v", "tcn.network.0.net.0.bias", "tcn.network.0.net.0.weight_g", "tcn.network.0.net.0.weight_v", "tcn.network.0.net.4.bias", "tcn.network.0.net.4.weight_g", "tcn.network.0.net.4.weight_v", "tcn.network.0.downsample.weight", "tcn.network.0.downsample.bias", "tcn.network.1.conv1.bias", "tcn.network.1.conv1.weight_g", "tcn.network.1.conv1.weight_v", "tcn.network.1.conv2.bias", "tcn.network.1.conv2.weight_g", "tcn.network.1.conv2.weight_v", "tcn.network.1.net.0.bias", "tcn.network.1.net.0.weight_g", "tcn.network.1.net.0.weight_v", "tcn.network.1.net.4.bias", "tcn.network.1.net.4.weight